僅須執行第一次

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install gradio
!pip install tensorflow
!pip install numpy
!pip install Pillow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile # Import the zipfile module
# 指定 .zip 檔案的路徑（請替換為您雲端硬碟的實際路徑）
zip_path = '/content/drive/MyDrive/archive.zip'  # 替換為您的檔案路徑
extract_path = '/content/drive/MyDrive/Colab Notebooks/dataset'  # 解壓縮目標資料夾

# 解壓 .zip 檔案
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"資料已解壓縮到 {extract_path}")

資料已解壓縮到 /content/drive/MyDrive/Colab Notebooks/dataset


每次都要執行

In [ ]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# Step 1: 定義資料路徑
train_path = "/content/drive/MyDrive/Colab Notebooks/dataset/flags/train"  # 訓練集路徑
test_path = "/content/drive/MyDrive/Colab Notebooks/dataset/flags/test"   # 測試集路徑
image_size = (224, 224)
batch_size = 32

# Step 2: 數據載入與增強
# 載入數據集
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=image_size,
    batch_size=batch_size
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_path,
    image_size=image_size,
    batch_size=batch_size
)
class_names = train_dataset.class_names
# 數據增強
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

def preprocess(image, label):
    image = data_augmentation(image)
    image = tf.cast(image, tf.float32) / 255.0  # 正規化到 [0, 1]
    return image, label

train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Step 3: 模型構建與 Fine-Tuning
def create_model(num_classes):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = True  # 解凍模型進行微調

    # 鎖定前一部分層
    for layer in base_model.layers[:-50]:  # 解凍最後 50 層
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax")  # num_classes 是國旗分類數
    ])
    return model

num_classes = len(class_names)  # Use the existing class_names variable

model = create_model(num_classes)

# 編譯模型
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


Found 19716 files belonging to 249 classes.
Found 5154 files belonging to 249 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Step 4: 配置模型檢查點保存
checkpoint_dir = "/content/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, "model_epoch_{epoch:02d}.keras")

# Import ModelCheckpoint explicitly
from tensorflow.keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    save_best_only=False,  # 保存所有 epoch 的模型
    save_freq="epoch",
    verbose=1
)

# 早停策略
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# 動態學習率調整
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6)



# Step 5: 開始訓練
initial_epoch = 0  # 如果之前有保存進度，這裡可以更改為上次保存的 epoch
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=20,  # 設置最大 epochs
    initial_epoch=initial_epoch,
    callbacks=[model_checkpoint, early_stopping, reduce_lr]
)

# Step 6: 繼續訓練（如果中斷）
# 加載保存的模型檢查點
from tensorflow.keras.models import load_model

# 指定要加載的檢查點
last_checkpoint = "/content/checkpoints/model_epoch_05.h5"  # 替換為最新的檢查點
model = load_model(last_checkpoint)

# 繼續訓練
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=20,  # 設置總的最大 epochs
    initial_epoch=5,  # 從第 5 個 epoch 繼續
    callbacks=[model_checkpoint, early_stopping, reduce_lr]
)

Epoch 1/20
617/617 ━━━━━━━━━━━━━━━━━━━━ 0s 628ms/step - accuracy: 0.0059 - loss: 9.9719
Epoch 1: saving model to /content/checkpoints/model_epoch_01.keras
617/617 ━━━━━━━━━━━━━━━━━━━━ 462s 700ms/step - accuracy: 0.0060 - loss: 9.9716 - val_accuracy: 0.0285 - val_loss: 9.4217 - learning_rate: 1.0000e-05
Epoch 2/20
616/617 ━━━━━━━━━━━━━━━━━━━━ 0s 616ms/step - accuracy: 0.0295 - loss: 9.3332
Epoch 2: saving model to /content/checkpoints/model_epoch_02.keras
617/617 ━━━━━━━━━━━━━━━━━━━━ 476s 685ms/step - accuracy: 0.0295 - loss: 9.3327 - val_accuracy: 0.0708 - val_loss: 8.7590 - learning_rate: 1.0000e-05
Epoch 3/20
616/617 ━━━━━━━━━━━━━━━━━━━━ 0s 612ms/step - accuracy: 0.0588 - loss: 8.7579
Epoch 3: saving model to /content/checkpoints/model_epoch_03.keras
617/617 ━━━━━━━━━━━━━━━━━━━━ 421s 681ms/step - accuracy: 0.0588 - loss: 8.7575 - val_accuracy: 0.1217 - val_loss: 8.1058 - learning_rate: 1.0000e-05
Epoch 4/20
616/617 ━━━━━━━━━━━━━━━━━━━━ 0s 596ms/step - accuracy: 0.0856 - loss: 8.2368


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/checkpoints/model_epoch_05.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# Step 6: 使用 Gradio 建立使用者介面
import gradio as gr
from PIL import Image

# **Use the stored class_names variable instead of trying to access it from train_dataset**
# class_names = list(train_dataset.class_names)  # This line is causing the error
# class_names is already available from previous cell execution

# 定義預測函數
def predict_flag(image):
    # 預處理圖片
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    # 模型預測
    predictions = model.predict(image)
    predicted_class = class_names[np.argmax(predictions)]
    confidence = np.max(predictions)

    return f"國旗: {predicted_class}, 置信度: {confidence:.2f}"



In [ ]:
# 建立 Gradio 介面
interface = gr.Interface(
    fn=predict_flag,
    inputs=gr.Image(type="pil", label="上傳國旗圖片"),
    outputs="text",
    title="國旗分類 AI",
    description="上傳國旗圖片，AI 將自動辨識國家的名稱。"
)

# 啟動介面
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54e5dc087405355396.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
